In [1]:
!pip install mlflow boto3 awscli optuna xgboost lightgbm imbalanced-learn

   ---------------------------------------- 0.0/383.6 kB ? eta -:--:--
   ----------- ---------------------------- 112.6/383.6 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 383.6/383.6 kB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.5 MB 6.1 MB/s eta 0:00:01
   -------------- ------------------------- 0.5/1.5 MB 5.4 MB/s eta 0:00:01
   ------------------ --------------------- 0.7/1.5 MB 4.7 MB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 4.0 MB/s eta 0:00:01
   -------------------------- ------------- 1.0/1.5 MB 4.0 MB/s eta 0:00:01
   --------------------------------- ------ 1.2/1.5 MB 4.2 MB/s eta 0:00:01
   ------------------------------------- -- 1.4/1.5 MB 4.1 MB/s eta 0:00:01
   -------------------------------------- - 1.4/1.5 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 3.5 MB/s eta 0:00:00


In [1]:
# Step 2: Set up mlflow tracking server
import mlflow

mlflow.set_tracking_uri("http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/")

In [2]:
# Set or create an experiment
mlflow.set_experiment("Exp 5: ML Algos with HP Tuning")

2025/04/09 14:50:03 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5: ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://yt-plugin-bucket/201394593778390276', creation_time=1744190402646, experiment_id='201394593778390276', last_update_time=1744190402646, lifecycle_stage='active', name='Exp 5: ML Algos with HP Tuning', tags={}>

In [3]:
import optuna 
import mlflow 
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [ ]:
df = pd.read_csv('../data/reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [5]:
# Step 1: Remap the class labesl from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1:2, 0:0, 1:1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3) # Trigram setting
max_features = 1000 # Set max_featues to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))

# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLFlow, passing the algo_name as "XGBoost"
    log_mlflow('XGBoost', best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost for XGBoost
run_optuna_experiment()


[I 2025-04-09 14:50:14,041] A new study created in memory with name: no-name-a4f9c614-8c48-41bb-bda5-61c9c25cd9fa
[I 2025-04-09 14:50:25,727] Trial 0 finished with value: 0.5548888585844811 and parameters: {'n_estimators': 79, 'learning_rate': 0.0015793564055546813, 'max_depth': 6}. Best is trial 0 with value: 0.5548888585844811.
[I 2025-04-09 14:51:11,928] Trial 1 finished with value: 0.579980908223101 and parameters: {'n_estimators': 246, 'learning_rate': 0.00012520666307915665, 'max_depth': 7}. Best is trial 1 with value: 0.579980908223101.
[I 2025-04-09 14:51:49,527] Trial 2 finished with value: 0.6772126005727533 and parameters: {'n_estimators': 203, 'learning_rate': 0.015592296623565038, 'max_depth': 7}. Best is trial 2 with value: 0.6772126005727533.
[I 2025-04-09 14:52:19,103] Trial 3 finished with value: 0.5861175507977635 and parameters: {'n_estimators': 115, 'learning_rate': 0.00024343509640108516, 'max_depth': 8}. Best is trial 2 with value: 0.6772126005727533.
[I 2025-04-0

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/201394593778390276/runs/14bc69e5b53945e792822c17b957c83b
🧪 View experiment at: http://ec2-13-61-19-60.eu-north-1.compute.amazonaws.com:5000/#/experiments/201394593778390276
